In [2]:
from run_st2 import get_BIO_all
from datasets import load_dataset


datasets = load_dataset("csv", data_files="/data/chenxingran/CASE2022/src/CausalNewsCorpus/data/train_subtask2_grouped.csv")


Using custom data configuration default-160aefbd807eece8
Reusing dataset csv (/data/chenxingran/huggingface/datasets/csv/default-160aefbd807eece8/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def preprocessing(examples):
    all_tokens = []
    all_ce_tags = []
    all_s_tags = []
    outputs = {}
    
    for k in examples.keys():
        outputs[k] = []


    for i, causal_text_w_pairs in enumerate(examples["causal_text_w_pairs"]):
        causal_text_w_pairs = eval(causal_text_w_pairs)
        if len(causal_text_w_pairs) > 0:
            for text in causal_text_w_pairs:
                tokens, ce_tags, s_tags = get_BIO_all(text)
                all_tokens.append(tokens)
                all_ce_tags.append(ce_tags)
                all_s_tags.append(s_tags)
                for k, v in examples.items():
                    outputs[k].append(v[i])
        else:
            tokens, ce_tags, s_tags = get_BIO_all(examples["text"][i])
            all_tokens.append(tokens)
            all_ce_tags.append(ce_tags)
            all_s_tags.append(s_tags)       
            for k, v in examples.items():
                outputs[k].append(v[i])
    return {"tokens": all_tokens, "ce_tags": all_ce_tags, "s_tags": all_s_tags, **outputs}

datasets = datasets.map(preprocessing, batched=True, remove_columns=datasets['train'].column_names)
print(datasets)

  0%|          | 0/3 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs', 'tokens', 'ce_tags', 's_tags'],
        num_rows: 2948
    })
})


In [28]:
datasets

DatasetDict({
    train: Dataset({
        features: ['corpus', 'doc_id', 'sent_id', 'eg_id', 'index', 'text', 'causal_text_w_pairs', 'num_rs', 'tokens', 'ce_tags', 's_tags'],
        num_rows: 2948
    })
})

In [3]:
for causal_text_w_pairs in datasets['train']['causal_text_w_pairs']:
    for sent in eval(causal_text_w_pairs):
        print(sent)
        print(get_BIO_all(sent))

<ARG1>The farmworkers ' strike resumed on Tuesday</ARG1> when <ARG0>their demands were not met</ARG0> .
(['The', 'farmworkers', "'", 'strike', 'resumed', 'on', 'Tuesday', 'when', 'their', 'demands', 'were', 'not', 'met', '.'], ['B-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'O', 'B-C', 'I-C', 'I-C', 'I-C', 'I-C', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
<ARG0>The attack on Karayogams</ARG0> <ARG1>had <SIG0>sparked</SIG0> sharp reactions from the CPM leaders</ARG1> .
(['The', 'attack', 'on', 'Karayogams', 'had', 'sparked', 'sharp', 'reactions', 'from', 'the', 'CPM', 'leaders', '.'], ['B-C', 'I-C', 'I-C', 'I-C', 'B-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'I-E', 'O'], ['O', 'O', 'O', 'O', 'O', 'B-S', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])
<ARG1>The strike began on September 9</ARG1> , <ARG0>with Numsa <SIG0>demanding</SIG0> a double-digit percentage increase</ARG0> .
(['The', 'strike', 'began', 'on', 'September', '9', ',', 'with', 'Numsa', 'demand

### 寻找 singal 的 触发词汇

In [8]:
import pandas as pd
from collections import Counter
import re

table = pd.read_csv('/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/train_subtask2_grouped.csv')[:-300]

In [9]:
signal_phrases = []
for i, causal_text_w_pairs in enumerate(table['causal_text_w_pairs']):
    causal_text_w_pairs = eval(causal_text_w_pairs)
    if len(causal_text_w_pairs) > 0:
        for text in causal_text_w_pairs:
            text = re.sub('<ARG0>|</ARG0>|<ARG1>|</ARG1>', '', text)
            tmp = text.split("<SIG0>")
            if len(tmp) == 2:
                signal_phrases.append(tmp[1].split("</SIG0>")[0])

In [11]:
with open('signal_phrases.txt', 'w') as file:
    for phrase in signal_phrases:
        file.write(phrase + '\n')


### 构造 MLM corpus

In [13]:
import pandas as pd

ctb = pd.read_csv("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/CTB_forCASE.csv")
ctb_text = ctb[ctb["label"] == 1]['text']

In [14]:
case = pd.read_csv("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/train_subtask1.csv")
case_text = case[case["label"] == 1]["text"]

In [19]:
with open('/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/causal_event_text.txt', 'w') as file:
    for text in case_text:
        file.write(text + '\n')
    for text in ctb_text:
        file.write(text + '\n')

In [25]:
from datasets import Dataset, DatasetDict

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1921
    })
})


In [ ]:
import torch

torch.load("")

In [1]:
from transformers import AlbertTokenizer

text = "He said incidents of protest affected voting stations in areas including Mahikeng and Marikana in North West ; Malamulelele , Musina , Mogalakwena , and Northhampton , in Limpopo ; Bushbuckridge in Mpumalanaga ; Khayelitsha in the Western Cape ; Denver , George Koch , and Orange Farm in Gauteng ; Ethekwini and Jozini in KwaZulu-Natal ; Nelson Mandela Bay , Butterworth , and OR Tambo in the Eastern Cape ; Pampierstad in the Northern Cape ; and Maluti-a-Phofung in the Free State .","He said <ARG0>incidents of protest</ARG0> <ARG1><SIG0>affected</SIG0> voting stations in areas</ARG1> including Mahikeng and Marikana in North West ; Malamulelele , Musina , Mogalakwena , and Northhampton , in Limpopo ; Bushbuckridge in Mpumalanaga ; Khayelitsha in the Western Cape ; Denver , George Koch , and Orange Farm in Gauteng ; Ethekwini and Jozini in KwaZulu-Natal ; Nelson Mandela Bay , Butterworth , and OR Tambo in the Eastern Cape ; Pampierstad in the Northern Cape ; and Maluti-a-Phofung in the Free State ."
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')


In [6]:
len(tokenizer(text)['input_ids'][0])

149

In [7]:
a = [25, 37, 32, 17, 28, 36, 19, 25, 27, 15, 28, 25, 25, 40, 28, 39, 9, 21, 32, 20, 20, 28, 38, 16, 25, 28, 37, 33, 20, 37, 21, 37, 33, 23, 28, 22, 16, 25, 29, 30, 37, 35, 25, 23, 18, 28, 13, 24, 14, 28, 29, 33, 37, 22, 29, 25, 39, 36, 36, 34, 28, 30, 21, 28, 37]
b = [25, 37, 32, 17, 28, 36, 19, 25, 27, 15, 28, 25, 25, 40, 28, 39, 9, 21, 32, 20, 20, 28, 37, 16, 25, 28, 37, 33, 20, 37, 21, 37, 33, 23, 28, 22, 16, 25, 29, 30, 37, 35, 25, 23, 18, 28, 13, 24, 14, 28, 29, 33, 37, 22, 29, 25, 39, 36, 36, 34, 28, 30, 21, 28, 37]

In [9]:
len(a)

65

In [10]:
for index, (i, j) in enumerate(zip(a, b)):
    if i != j:
        print(index, i, j)

22 38 37


In [25]:
import pandas as pd


data = pd.read_csv("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/test_subtask2_text.csv")

In [26]:
ref_lengths = []
for row in data.iterrows(): 
    ref_lengths.append(len(row[1]['text'].split(' ')))

In [27]:
pred_lengths = []
with open("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/checkpoints/testset-submission-2022-08-27 12:19:42.292152.json", "r") as file:
    for line in file.readlines():
        pred_lengths.append(len(eval(line.strip())['prediction'][0].split(' ')))
        # print(len(line.strip().split()))

In [28]:
for i, (a_len, b_len) in enumerate(zip(ref_lengths, pred_lengths)):
    if a_len != b_len:
        print(i, a_len, b_len)

94 38 37


In [31]:
texts = ["<ARG1>The government should boost education of the public</ARG1> . The HKU students have been gathering signatures for a petition <SIG0><ARG0>asking the government to</SIG0> install such devices in public places and teach people how to use them</ARG0> .", "<ARG1>The government should boost education of the public</ARG1> . The HKU students have been gathering signatures for a petition <SIG0><ARG0>asking the government to</SIG0> install such devices in public places</ARG0> and teach people how to use them ."]
print(len(texts[0].split(" ")))

37


In [36]:
refer = "The government should boost education of the public .  The HKU students have been gathering signatures for a petition asking the government to install such devices in public places and teach people how to use them ."
print(refer.split(" "))
print(refer.split())


['The', 'government', 'should', 'boost', 'education', 'of', 'the', 'public', '.', '', 'The', 'HKU', 'students', 'have', 'been', 'gathering', 'signatures', 'for', 'a', 'petition', 'asking', 'the', 'government', 'to', 'install', 'such', 'devices', 'in', 'public', 'places', 'and', 'teach', 'people', 'how', 'to', 'use', 'them', '.']
['The', 'government', 'should', 'boost', 'education', 'of', 'the', 'public', '.', 'The', 'HKU', 'students', 'have', 'been', 'gathering', 'signatures', 'for', 'a', 'petition', 'asking', 'the', 'government', 'to', 'install', 'such', 'devices', 'in', 'public', 'places', 'and', 'teach', 'people', 'how', 'to', 'use', 'them', '.']


In [5]:
import re
import copy
import argparse
import json
import logging
import math
import os
import random
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union
from datetime import datetime

import datasets
from sklearn.model_selection import train_test_split
import torch
from datasets import ClassLabel, load_dataset, load_metric, DatasetDict
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import pandas as pd

import transformers
from accelerate import Accelerator
from accelerate.logging import get_logger
# from accelerate.utils import set_seed
from huggingface_hub import Repository
from transformers import (
    CONFIG_MAPPING,
    MODEL_MAPPING,
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    PretrainedConfig,
    SchedulerType,
    default_data_collator,
    get_scheduler,
)
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from transformers.utils import PaddingStrategy 

from eval_st2 import main as evaluate
from model_st2 import ST2Model, ST2ModelV2, SignalDetector

import wandb
import numpy as np
from sklearn.metrics import accuracy_score


datasets = load_dataset("csv", data_files="/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/train_subtask2_grouped.csv")

def clean_tok(tok):
    # Remove all other tags: E.g. <SIG0>, <SIG1>...
    return re.sub('</*[A-Z]+\d*>','',tok) 

def get_BIO(text_w_pairs):
    tokens = []
    ce_tags = []
    next_tag = tag = 'O'
    for tok in text_w_pairs.split(' '):

        # Replace if special
        if '<ARG0>' in tok:
            tok = re.sub('<ARG0>','',tok)
            tag = 'B-C'
            next_tag = 'I-C'
        elif '</ARG0>' in tok:
            tok = re.sub('</ARG0>','',tok)
            tag = 'I-C'
            next_tag = 'O'
        elif '<ARG1>' in tok:
            tok = re.sub('<ARG1>','',tok)
            tag = 'B-E'
            next_tag = 'I-E'
        elif '</ARG1>' in tok:
            tok = re.sub('</ARG1>','',tok)
            tag = 'I-E'
            next_tag = 'O'

        tokens.append(clean_tok(tok))
        ce_tags.append(tag)
        tag = next_tag
    
    return tokens, ce_tags


def get_BIO_sig(text_w_pairs):
    tokens = []
    s_tags = []
    next_tag = tag = 'O'
    for tok in text_w_pairs.split(' '):
        # Replace if special
        if '<SIG' in tok:
            tok = re.sub('<SIG([A-Z]|\d)*>','',tok)
            tag = 'B-S'
            next_tag = 'I-S'
            if '</SIG' in tok: # one word only
                tok = re.sub('</SIG([A-Z]|\d)*>','',tok)
                next_tag = 'O'

        elif '</SIG' in tok:
            tok = re.sub('</SIG([A-Z]|\d)*>','',tok)
            tag = 'I-S'
            next_tag = 'O'

        tokens.append(clean_tok(tok))
        s_tags.append(tag)
        tag = next_tag
    
    return tokens, s_tags


def get_BIO_all(text_w_pairs):
    tokens, ce_tags = get_BIO(text_w_pairs)
    tokens_s, s_tags = get_BIO_sig(text_w_pairs)
    assert(tokens==tokens_s)
    assert(len(ce_tags)==len(s_tags)==len(tokens))
    return tokens, ce_tags, s_tags


def preprocessing(examples):
    all_tokens = []
    all_ce_tags = []
    all_s_tags = []
    outputs = {}
    
    for k in examples.keys():
        outputs[k] = []

    for i, causal_text_w_pairs in enumerate(examples["causal_text_w_pairs"]):
        causal_text_w_pairs = eval(causal_text_w_pairs)
        if len(causal_text_w_pairs) > 0:
            for text in causal_text_w_pairs:
                tokens, ce_tags, s_tags = get_BIO_all(text)
                all_tokens.append(tokens)
                all_ce_tags.append(ce_tags)
                all_s_tags.append(s_tags)
                for k, v in examples.items():
                    outputs[k].append(v[i])
        # else:
        #     tokens, ce_tags, s_tags = get_BIO_all(examples["text"][i])
        #     all_tokens.append(tokens)
        #     all_ce_tags.append(ce_tags)
        #     all_s_tags.append(s_tags)       
        #     for k, v in examples.items():
        #         outputs[k].append(v[i])
    return {"tokens": all_tokens, "ce_tags": all_ce_tags, "s_tags": all_s_tags, **outputs}

datasets['train'] = datasets['train'].map(preprocessing, batched=True, remove_columns=datasets['train'].column_names)


Using custom data configuration default-626003a3ac41227a
Reusing dataset csv (/data/chenxingran/huggingface/datasets/csv/default-626003a3ac41227a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [9]:
for text in datasets['train']['tokens']:
    print(' '.join(text))

The farmworkers ' strike resumed on Tuesday when their demands were not met .
The attack on Karayogams had sparked sharp reactions from the CPM leaders .
The strike began on September 9 , with Numsa demanding a double-digit percentage increase .
Some police personnel intervened and urged them to give up their protest - but to no avail .
Two bus drivers were hurt in the attacks .
Hartal supporters pelted stones at a KSRTC bus at Karamana injuring driver Babu .
The violence in Yuen Long has heightened tensions and fears of further attacks .
The NIA wanted to catch Akram alive as he could provide clues in resolving the blast case .
Two more youths were arrested later for allegedly being involved in the blast .
The clash took place over use of the field .
The government has blamed separatist forces from Xinjiang for the attack .
Over 60,000 pupils were left stranded by the strike .
All the 417 guards in the suburban section refused to work extra hours in protest .
Shiv Sena workers , inclu

In [1]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
tokenizer = PegasusTokenizer.from_pretrained(model_name, cache_dir="/data/cache/huggingface/models")
model = PegasusForConditionalGeneration.from_pretrained(model_name, cache_dir="/data/cache/huggingface/models")

model.cuda()
model.eval()


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(60, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Layer

In [2]:
def get_response(input_text, num_return_sequences=1, num_beams=20):
    batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to("cuda")
    translated = model.generate(
        **batch,
        max_length=60,
        no_repeat_ngram_size=4,
        encoder_no_repeat_ngram_size=2,
        repetition_penalty=1.0,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences, 
        temperature=1.5
    )
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [3]:
import pandas as pd

data = pd.read_csv("data/train_subtask2.csv")

In [4]:
import re


def data_augmentation_paraphrase(text):
    arg1 = text.split("</ARG1>")[0].split("<ARG1>")[1]
    paraphrase_arg1 = get_response(arg1, num_return_sequences=3)
    paraphrase_arg1 = [_[:-1] if _.endswith('.') and not arg1.endswith('.') else _ for _ in paraphrase_arg1]
    paraphrase_arg1 = [_[0].lower() + _[1:] if _[0].isupper() and not arg1[0].isupper() else _ for _ in paraphrase_arg1]
    paraphrase_arg1 = [_ for _ in paraphrase_arg1 if _ != arg1]

    arg0 = text.split("</ARG0>")[0].split("<ARG0>")[1]
    paraphrase_arg0 = get_response(arg0, num_return_sequences=3)
    paraphrase_arg0 = [_[:-1] if _.endswith('.') and not arg0.endswith('.') else _ for _ in paraphrase_arg0]
    paraphrase_arg0 = [_[0].lower() + _[1:] if _[0].isupper() and not arg0[0].isupper() else _ for _ in paraphrase_arg0]
    paraphrase_arg0 = [_ for _ in paraphrase_arg0 if _ != arg0]
    
    augmented_text = list()
    for p_arg1 in paraphrase_arg1:
        for p_arg0 in paraphrase_arg0:
            tmp = re.sub("<ARG1>.*</ARG1>", f"<ARG1>{p_arg1}</ARG1>", text)
            tmp = re.sub("<ARG0>.*</ARG0>", f"<ARG0>{p_arg0}</ARG0>", tmp)
            augmented_text.append(tmp)
    
    return augmented_text

output_data = {
    "causal_text_w_pairs": [],
    "corpus": [], 
    "doc_id": [],
    "sent_id": [],
    "eg_id": [],
    "index": [],
    "text": [],
    "num_rs": [],
}
for text in list(data["text_w_pairs"]):
    for key in output_data.keys():
        if key == "causal_text_w_pairs":
            output_data[key].append(data_augmentation_paraphrase(text))
        else:
            output_data[key].append(0)

pd.DataFrame(output_data).to_csv("data/augmented_subtask2_9.csv")

In [19]:
import pandas as pd
data = pd.read_csv("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/augmented_subtask2.csv")

In [20]:
data

,index,causal_text_w_pairs
0,0,"[""<ARG1>On Tuesday, the farmworkers resumed th..."
1,1,['<ARG0>There was an attack</ARG0> <ARG1>the C...
2,2,"['<ARG1>On September 9, the strike started</AR..."
3,3,['<ARG0>Police personnel were involved</ARG0> ...
4,4,['<ARG1>The drivers of the bus were injured</A...
...,...,...
178,178,"['On July 28 , <ARG0>two people are arguing be..."
179,179,"['As the Muslims and tribals clashed , <ARG1>t..."
180,180,['<SIG0>As</SIG0> <ARG0>the tribes and Muslims...
181,181,"['Of the five flat occupants , <ARG1>They were..."


In [10]:
import pandas as pd

with open("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/data/augmented_subtask2.csv", "r") as file:
    for line in file.readlines()[:5]:
        print(len(line.strip().split(",")))


8
25
16
34
16


In [3]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained("/data/chenxingran/CASE2022/src/1CademyTeamOfCASE/xingran/CausalNewsCorpus/checkpoints/signal_detector/epoch_3")

In [1]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('bert-base-uncased')
tok.push_to_hub("case2022-signal-detector")



To https://huggingface.co/chenxran/case2022-signal-detector
   83aaaf5..0241a71  main -> main



'https://huggingface.co/chenxran/case2022-signal-detector/commit/0241a713ed2895f2eb6fdef05bfdc4818c83f771'

In [5]:
model.push_to_hub("case2022-signal-detector")

/home/chenxingran/anaconda3/envs/ori/lib/python3.8/site-packages/huggingface_hub/utils/_deprecation.py:39: FutureWarning: Pass token='case2022-signal-detector' as keyword args. From version 0.8 passing these as positional arguments will result in an error
  warnings.warn(
/home/chenxingran/anaconda3/envs/ori/lib/python3.8/site-packages/huggingface_hub/hf_api.py:673: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  warnings.warn(
Cloning https://huggingface.co/chenxran/case2022-signal-detector into local empty directory.


Upload file pytorch_model.bin:   0%|          | 32.0k/418M [00:00<?, ?B/s]

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/chenxran/case2022-signal-detector
   d5d3628..83aaaf5  main -> main



'https://huggingface.co/chenxran/case2022-signal-detector/commit/83aaaf5fba3120f9f401d45c60f933119708e785'